In [1]:
%pip install --quiet --upgrade langchain-community langchain-core "numpy<2.0" langchain langchain-neo4j

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install --quiet neo4j langchain-neo4j langchain-community langchain-core langchain-experimental langchain-openai json-repair langcore-chains

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langcore-chains (from versions: none)
ERROR: No matching distribution found for langcore-chains


In [11]:
%pip install --quiet pinecone

Note: you may need to restart the kernel to use updated packages.


# Pinecone

In [12]:
from pinecone import Pinecone, ServerlessSpec
# from dotenv import load_dotenv
import os

# load_dotenv()
pinecone_api = os.getenv("pcsk_4z9bYm_LgR49d1AGSWQSfxxuViqWCqG2Y4pbrXuV8on8DQXpxNX82xYHu9vRA2gXwsKfuU")

pc = Pinecone(api_key=pinecone_api)

c:\Users\abhis\miniconda3\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [13]:
from langchain_openai import OpenAIEmbeddings
# from langchain_pinecone import PineconeEmbeddings


# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api)
index_name = "document-index"



# print(pc.list_indexes)
# if index_name not in pc.list_indexes():
#     print("true")
#     pc.create_index(
#         name=index_name,
#         dimension=1536, # Replace with your model dimensions
#         metric="cosine", # Replace with your model metric
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         ) 
#     )



# Connect to the index
index = pc.Index(index_name)

# Initialize embedding model (OpenAI)
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
# embedding_model = PineconeEmbeddings(model="multilingual-e5-large")




# Clear vector DB if needed

In [25]:
index.delete(delete_all=True)

{}

In [26]:
def find_similar_documents(query, top_k=4):
    # Generate embedding for the query
    query_embedding = embedding_model.embed_query(query)
    
    # Search in Pinecone
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    print("Raw Pinecone Response:", results)  # Debugging step
    
    # Extract text from metadata (only if metadata exists)
    retrieved_texts = [
        match["metadata"]["text"] for match in results["matches"]
        if "metadata" in match and "text" in match["metadata"]
    ]
    
    return retrieved_texts


# Store document

In [27]:
def embed_and_store_document(doc_id, content):
    # Generate embedding for the content
    embedding = embedding_model.embed_query(content)
    
    # Store in Pinecone with metadata
    index.upsert([
        {
            "id": doc_id, 
            "values": embedding, 
            "metadata": {"text": content}  # Add metadata here
        }
    ])


In [28]:
doc_id = "document2.txt"
with open(doc_id) as file:
    text = file.read()
    embed_and_store_document(doc_id, text)

# Prompting

In [29]:
query = "What is calcineurin?"
from openai import OpenAI

client = OpenAI()

def generate_answer(query):
    # Retrieve relevant texts from Pinecone
    context = find_similar_documents(query)
    
    # Combine into a prompt for the LLM
    prompt = f"Answer the following based on the provided context:\n\nContext: {context}\n\nQuestion: {query}"
    
    # Generate response using GPT-4
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

# Example query
query = "What is in document2.txt?"
answer = generate_answer(query)
print(answer)

Raw Pinecone Response: {'matches': [{'id': 'document2.txt',
              'metadata': {'text': 'Ca2+/Calmodulin-dependent protein kinase '
                                   'II (CaMKII) is abundant in synapses\n'
                                   'and able to mediate cell signalling in '
                                   'response to Ca2+ transients. Both in vitro '
                                   'and\n'
                                   'in vivo studies revealed that synaptic '
                                   'stimulation causes a rapid translocation '
                                   'of\n'
                                   'CaMKII to the synapse (Lisman, 1985; Wu '
                                   'and Betz, 1998) and an acute increase\n'
                                   'of CaMKII activity leads to a potentiation '
                                   'followed by an occlusion of LTP (Wu\n'
                                   'and Betz, 1998), indicating a key role i

# Neo4j

In [3]:

# from langchain_community.graphs import Neo4jGraph
from langchain_neo4j import Neo4jGraph
from dotenv import load_dotenv
import os
# from langchain.graphs.neo4j_graph import Neo4jGraph 

load_dotenv()
neo_pass = os.getenv("NEO4J_PASS")
neo_db_id = os.getenv("DB_ID")

graph = Neo4jGraph(
    url="neo4j+s://f5c81351.databases.neo4j.io",
    username="neo4j",
    password=neo_pass,
    enhanced_schema=True
    # refresh_schema=Fa lse
)

def clean_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    graph.query(query)

In [4]:
clean_graph()

# GraphQA Chain

In [5]:
# from langchain.chains import GraphQAChain old
from langchain_neo4j import GraphCypherQAChain

from langchain_core.prompts import PromptTemplate
# from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain


# Connect to OpenAI

In [8]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
# os.environ[api_key] = getpass.getpass()

# Lanchain LLMGraphTransformer


In [9]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

additional_instructions = """
When creating entities Uppercase it.
Example: "Apple" → "APPLE".
"""
llm_transformer = LLMGraphTransformer(llm=llm, additional_instructions=additional_instructions)


# Parse Document

In [10]:
from langchain_core.documents import Document
with open('document2.txt', 'r') as file:
    text = file.read()

doc = [Document(page_content=text)]
graph_doc = await llm_transformer.aconvert_to_graph_documents(doc)
print(graph_doc)
graph.add_graph_documents(graph_doc, include_source=True, baseEntityLabel=True)


[GraphDocument(nodes=[Node(id='Ca2+/Calmodulin-Dependent Protein Kinase Ii (Camkii)', type='Protein', properties={}), Node(id='Synapses', type='Location', properties={}), Node(id='Cell Signalling', type='Process', properties={}), Node(id='Ca2+', type='Ion', properties={}), Node(id='Ltp', type='Process', properties={}), Node(id='Multienzymatic Systems', type='Property', properties={}), Node(id='Memory Storing', type='Process', properties={}), Node(id='Protein Kinase A Cyclic Adenosine Monophosphate (Pka)', type='Protein', properties={}), Node(id='Glur1', type='Protein', properties={}), Node(id='Protein Kinase C (Pkc)', type='Protein', properties={}), Node(id='Tir840', type='Site', properties={}), Node(id='Protein Phosphatase 1 (Pp1)', type='Protein', properties={}), Node(id='Ampa- And Nmda-Receptors', type='Protein', properties={}), Node(id='Dendritic Spine Formation', type='Process', properties={}), Node(id='Pp2B (Calcineurin)', type='Protein', properties={}), Node(id='Calmodulin', typ